In [ ]:
'''
Vous devez constituer des équipes de 4 étudiants.
Vous serez évalués sur un rendu sur Github.
L'objet du projet est soit d'entraîner un classifieur VIT sur un dataset d'images labélisées (différent de ceux vus en TP et de ceux choisis par les autres équipes),
soit d'entrainer/fine-tuner un modèle de diffusion (par exemple lucidrain) sur un dataset qui vous intéresse (mangas, joueurs de foot, ...).
Dans le readme vous décrirez comment vous avez constitué votre dataset,
et donnerez des exemples d'images du dataset/images générées par votre modèle (dans le cas génératif)
ainsi que la fonction permettant de sampler de nouvelles images, ou bien l'accuracy dans le cas de classifieur
(avec une fonction indiquant comment récupérer le dataset de validation et lancer le calcul de la métrique (afin que nous puissions vérifier)).
'''
!pip install Augmentor

import torch # https://pytorch.org/docs/stable/index.html
import torchvision # https://pytorch.org/vision/stable/index.html
import os
import shutil
import Augmentor

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)
print(torch.cuda.get_device_name())

In [ ]:
img_size = 250
channel_size = 3
transform = []
transform_list = [
        torchvision.transforms.Resize((img_size,img_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=0.5, std=0.5),
        torchvision.transforms.RandomHorizontalFlip(p=0.5),
        torchvision.transforms.RandomVerticalFlip(p=0.5)
        ]
if channel_size == 1 :
    transform_list.append(torchvision.transforms.Grayscale())

transform = torchvision.transforms.Compose(transform_list)
print(transform)

In [ ]:
'''
Le code suivant est censé devoir ajouter des nouvelles images pour pouvoir avoir des meilleures résultats dans l'entrainement du model,
mais ça ne fonctionne pas, les images ne sont pas sauvegardés dans le dossier spécifié.

import os
from torchvision import transforms
import torchvision.datasets as datasets
import Augmentor

# Définir le chemin d'entrée pour les données originales
input_path = "/content/data/oxford-iiit-pet/images"

# Définir le chemin de sortie pour les images générées
output_path = "/content/data/oxford-iiit-pet/images/output"

# Créer un pipeline Augmentor avec le chemin de sortie spécifié
p = Augmentor.Pipeline(input_path, output_directory=output_path)

# Ajouter des opérations d'augmentation
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.flip_left_right(probability=0.5)
p.flip_top_bottom(probability=0.5)

# Spécifier le format d'enregistrement des images (PNG)
p.set_save_format("PNG")

# Spécifier le nombre d'images que vous souhaitez générer
num_images_to_generate = 1

# Générer de nouvelles images
p.sample(num_images_to_generate)

# Processus d'écriture des images dans le dossier de sortie spécifié
p.process()

# Créer un transform similaire à celui utilisé pour les données d'entraînement
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Charger les nouveaux jeux de données d'entraînement et de test à partir des images augmentées
train_dataset = datasets.ImageFolder(root=output_path, transform=transform)
test_dataset = datasets.OxfordIIITPet(root='./data/', split="test", transform=transform, download=True)

# Afficher le nombre total d'images dans le nouvel ensemble de données d'entraînement
print('Nombre total d\'images dans l\'ensemble de données d\'entraînement:', len(train_dataset))
'''

In [ ]:
train_dataset = torchvision.datasets.OxfordIIITPet(root='./data/',split="trainval", transform=transform,download=True)
test_dataset = torchvision.datasets.OxfordIIITPet(root='./data/',split="test", transform=transform,download=True)
print('Nombre d\'images d\'entraînement:', len(train_dataset))
print('Nombre d\'image de test:', len(test_dataset))

In [18]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader =torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)

In [ ]:
examples = enumerate(test_dataloader)
batch_idx, data = next(examples)
print(len(data))


In [ ]:
print('Images:', data[0].shape)
b, c, h, w = data[0].shape
print('Batch(s):', b)
print('Channel(s):', c)
print('Height:', h)
print('Width:', w)

In [ ]:
print('Labels:', data[1].shape) # 32 labels
print(data[1])
classes = train_dataset.classes

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6 if b > 6 else b):
    plt.subplot(2, 3, i+1)
    plt.tight_layout()
    plt.imshow(data[0][i][0], cmap="gray" ,interpolation='none')
    plt.title(f'Label: {classes[data[1][i]]}')
    plt.xticks([])
    plt.yticks([])

In [ ]:
from TP5_MHA import MultiHeadAttention
class VisionEncoder(torch.nn.Module):
    def __init__(self, embed_size, num_heads, dropout):
        super(VisionEncoder, self).__init__()

        self.embed_size = embed_size
        self.num_heads = num_heads
        self.dropout = dropout

        self.norm1 = torch.nn.LayerNorm(self.embed_size)
        self.norm2 = torch.nn.LayerNorm(self.embed_size)
        self.attention = MultiHeadAttention(self.embed_size, self.num_heads, self.dropout)
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(embed_size, embed_size*4),
            torch.nn.GELU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(embed_size*4, embed_size),
            torch.nn.Dropout(0.2)
            )

    def forward(self, x):
        # x.expand(32,-1,-1)
        fwd_norm1 = self.norm1(x)
        fwd_attention = fwd_norm1 + self.attention(fwd_norm1, fwd_norm1, fwd_norm1)
        fwd_norm2 = self.norm2(fwd_attention)
        fwd_mlp = fwd_attention + self.mlp(fwd_norm2)

        return fwd_mlp

In [ ]:
class ViT(torch.nn.Module):
    class_token = []
    positional_encoding = []
    def __init__(self, image_size, channel_size, patch_size, embed_size, nb_heads, classes, nb_layers, hidden_size, dropout):
        super(ViT, self).__init__()

        self.patch_size = patch_size
        self.embed_size = embed_size
        self.nb_patches = (image_size // patch_size) ** 2
        self.pixels_per_patch = channel_size * (patch_size ** 2)
        self.nb_heads = nb_heads
        self.classes = classes
        self.nb_layers = nb_layers
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.class_token = torch.rand(1, 1, self.embed_size).to(device)
        self.positional_encoding = torch.randn(1,self.nb_patches + 1,self.embed_size).to(device)
        torch.nn.Parameter(self.class_token)
        torch.nn.Parameter(self.positional_encoding)

        self.embedding = torch.nn.Linear(self.pixels_per_patch,embed_size)
        self.dropout_layer = torch.nn.Dropout(dropout)
        self.encoders = torch.nn.ModuleList([])
        for i in range(self.nb_layers):
            self.encoders.append(VisionEncoder(self.embed_size, self.nb_heads, self.dropout))

        self.norm = torch.nn.LayerNorm(self.embed_size)
        self.classifier = torch.nn.Linear(self.embed_size, self.classes)


    def forward(self, img_torch):
        b, c, h, w = img_torch.size()
        img_torch_reshape = img_torch.reshape(b, int((h / self.patch_size) * (w / self.patch_size)), c * self.patch_size * self.patch_size)
        fwd_embeddings = self.embedding(img_torch_reshape)
        self.class_token = self.class_token.expand(b, 1, self.embed_size).to(device)
        fwd_cat_class_token = torch.cat((fwd_embeddings, self.class_token),1)

        fwd_pos_encoding = fwd_cat_class_token + self.positional_encoding
        fwd_dropout = self.dropout_layer(fwd_pos_encoding)

        for encoder in self.encoders:
            fwd_dropout = encoder(fwd_dropout)

        fwd_cls = fwd_dropout[:, -1]
        fwd_cls = self.norm(fwd_cls)
        fwd_cls = self.classifier(fwd_cls)
        fwd_softmax = torch.nn.functional.log_softmax(fwd_cls, -1)
        return fwd_softmax

model = ViT(image_size=img_size, channel_size=channel_size, patch_size=25, embed_size=512, nb_heads=8, classes=len(classes), nb_layers=6, hidden_size=256, dropout=0.2).to(device)
print(model)



In [ ]:

loss_fct = torch.nn.NLLLoss()
print(loss_fct)
optimizer = torch.optim.Adam(model.parameters(), lr= 5e-5)
print(optimizer)

In [ ]:
losses = []
accuracies = []
nb_epochs = 10
def train_model():
    for epoch in range(nb_epochs):
        model.train()
        epoch_loss = 0
        y_pred = []
        y_true = []

        for batch_idx, (imgs, labels) in enumerate(train_dataloader):
            imgs = imgs.to(device)
            labels = labels.to(device)
            predictions = model(imgs)
            loss = loss_fct(predictions, labels)
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()


            y_pred.extend(predictions.argmax(dim=1).tolist())  # Compléter ici (indice : on veut l'indice de la valeur maximale des éléments du tenseur pour chaque batch, une fonction PyTorch existe pour cela !)
            y_true.extend(labels.tolist())

            # Ajout de la valeur de loss du batch à la valeur de loss sur l'ensemble de l'epoch
            epoch_loss += loss.item()

        # Ajout de la loss de l'epoch à la liste de l'ensemble des loss
        losses.append(epoch_loss)

        # Vérification et calcul de la précision du modèle en comparant pour chaque image son label avec la valeur prédite
        nb_imgs = len(y_pred)
        total_correct = 0
        for i in range(nb_imgs):
            if y_pred[i] == y_true[i]:
                total_correct += 1
        accuracy = total_correct * 100 / nb_imgs

        # Ajout de la précision à la liste des précisions
        accuracies.append(accuracy)

        # Affichage des résultats pour l'epoch en cours (loss et précision)
        print("----------")
        print("")
        print("Epoch:", epoch)
        print("")
        print("Loss:", epoch_loss)
        print("")
        print(f"Accuracy: {accuracy} % ({total_correct} / {nb_imgs})")
        print("")
train_model()

In [ ]:
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Train loss")
plt.show()

In [ ]:
plt.plot(accuracies)
plt.xlabel("Epoch")
plt.ylabel("Train accuracy (%)")
plt.show()

In [ ]:
def eval_model():
    with torch.no_grad():
        model.eval()

        y_test_pred = []
        y_test_true = []

        print(test_dataloader)
        for batch_idx, (imgs, labels) in enumerate(test_dataloader):
            imgs = imgs.to(device)
            labels = labels.to(device)
            if(torch.Size([b]) == labels.size()):
                predictions = model(imgs)
                y_test_pred.extend(predictions.argmax(dim=1).tolist())
                y_test_true.extend(labels.tolist())

        nb_imgs = len(y_test_pred)
        total_correct = 0
        for i in range(nb_imgs):
            if y_test_pred[i] == y_test_true[i]:
                total_correct += 1
        accuracy = total_correct * 100 / nb_imgs

        print(f"Evaluation accuracy: {accuracy} % ({total_correct} / {nb_imgs})")
eval_model()